Laden eines ZIP-Archiv mit dem Drohnen-Datensatz aus dem angegebenen GitHub-Repository und entpacken in das aktuelle Arbeitsverzeichnis

In [ ]:
!curl -L -o drones_dataset.zip https://github.com/SmartFarmingLab/drones_seminar_dataset/archive/refs/heads/main.zip
!unzip drones_dataset.zip

In [ ]:
%pip install torch torchvision
%pip install matplotlib

Logischer Split in train, test, validation

In [1]:
import json, random
from pathlib import Path

# === Pfade anpassen ===
ANNOT_PATH = Path("data/tiles_2048/Weißenfelser Straße (250410PeO)/instances_all.json")
OUT_DIR = Path("data/tiles_2048/Weißenfelser Straße (250410PeO)")

# === Parameter ===
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}
RNG_SEED = 42

# === JSON laden ===
with open(ANNOT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

images = data["images"]
anns = data.get("annotations", [])
cats = data.get("categories", [])

# === Split zufällig berechnen ===
random.seed(RNG_SEED)
ids = [im["id"] for im in images]
random.shuffle(ids)
n = len(ids)
n_train = int(SPLITS["train"] * n)
n_val = int(SPLITS["val"] * n)
train_ids = set(ids[:n_train])
val_ids   = set(ids[n_train:n_train+n_val])
test_ids  = set(ids[n_train+n_val:])

def subset(ids_set):
    return {
        "images": [im for im in images if im["id"] in ids_set],
        "annotations": [a for a in anns if a.get("image_id") in ids_set],
        "categories": cats
    }

# === Drei JSON-Dateien speichern ===
for name, ids_set in [("train", train_ids), ("val", val_ids), ("test", test_ids)]:
    out_path = OUT_DIR / f"annotations_{name}.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(subset(ids_set), f, ensure_ascii=False, indent=2)
    print(f"→ {out_path} gespeichert")

print("Fertig.")


→ data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_train.json gespeichert
→ data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_val.json gespeichert
→ data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_test.json gespeichert
Fertig.



Definiert die Klasse CocoDataset, die COCO-formatierte Datensätze (Bilder + Annotationen) lädt.

Liest die JSON-Annotationen und ordnet sie den jeweiligen Bildern zu.

Lädt Bilder, konvertiert sie in RGB und wendet optionale Transformationen an.

Erstellt für jedes Bild ein Target-Dictionary mit image_id, boxes (Bounding Boxes) und labels.

Gibt in __getitem__ ein Bild-Tensor und das zugehörige Target zurück.

Definiert eine collate_fn, um Batches mit variabler Target-Länge korrekt zusammenzustellen.

In [2]:
from pathlib import Path
import json
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from collections import defaultdict

class CocoDataset(Dataset):
    def __init__(self, images_dir, ann_path, transform=None):
        self.images_dir = Path(images_dir)
        with open(ann_path, "r", encoding="utf-8") as f:
            d = json.load(f)
        self.images = d["images"]
        self.anns = d.get("annotations", [])
        self.categories = d.get("categories", [])
        self.transform = transform or T.ToTensor()

        # Index: image_id -> list(annotations)
        by_img = defaultdict(list)
        for a in self.anns:
            by_img[a["image_id"]].append(a)
        self.by_img = by_img

        # Indexliste
        self.records = [(im["id"], im["file_name"]) for im in self.images]

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        image_id, file_name = self.records[idx]
        img = Image.open(self.images_dir / file_name).convert("RGB")

        anns = self.by_img.get(image_id, [])
        boxes, labels = [], []
        for a in anns:
            if "bbox" in a:  # COCO: [x, y, w, h]
                x, y, w, h = a["bbox"]
                boxes.append([x, y, x + w, y + h])
            if "category_id" in a:
                labels.append(a["category_id"])

        target = {
            "image_id": torch.tensor([image_id], dtype=torch.int64),
            "boxes": torch.tensor(boxes, dtype=torch.float32) if boxes else torch.zeros((0,4), dtype=torch.float32),
            "labels": torch.tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,), dtype=torch.int64),
        }

        img = self.transform(img)  # Tensor [C,H,W]
        return img, target

# Für variable Target-Längen (Detection) braucht der DataLoader ein einfaches collate_fn:
def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    return list(imgs), list(targets)


Definiert Pfade zu den Bild- und Annotationsdateien für Training, Validierung und Test.

Erstellt für jede Datenteilmenge ein CocoDataset-Objekt.

Initialisiert DataLoader für Training, Validierung und Test, um die Daten in Batches zu laden.

Verwendet collate_fn, um variable Target-Größen (z. B. unterschiedliche Objektanzahlen pro Bild) korrekt zu verarbeiten.

In [3]:
IMAGES_DIR = "data/tiles_2048/Weißenfelser Straße (250410PeO)"
ANN_TRAIN = "data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_train.json"
ANN_VAL   = "data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_val.json"
ANN_TEST  = "data/tiles_2048/Weißenfelser Straße (250410PeO)/annotations_test.json"

train_ds = CocoDataset(IMAGES_DIR, ANN_TRAIN)
val_ds   = CocoDataset(IMAGES_DIR, ANN_VAL)
test_ds  = CocoDataset(IMAGES_DIR, ANN_TEST)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,  num_workers=0, collate_fn=collate_fn)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=0, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=4, shuffle=False, num_workers=0, collate_fn=collate_fn)



Initialisiert Faster R-CNN (ResNet50-FPN v2), ersetzt den Box-Kopf für num_classes_fg + 1 Klassen und verschiebt alles aufs passende Device.

Richtet AdamW (LR=1e-4, Weight Decay 1e-4) und Mixed-Precision (AMP mit GradScaler) ein.

Trainingsschleife über EPOCHS: lädt Batches, verschiebt Daten aufs Device, berechnet Loss im autocast-Kontext, führt Backprop mit skaliertem Loss aus und optimiert die Gewichte.

Protokolliert Schritt- und Epochenverluste; misst Dauer pro Epoche.

Speichert nach jeder Epoche die Modellgewichte unter models/model_epoch_{epoch}.pth.

In [5]:
import torch
import time
import matplotlib.pyplot as plt
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.optim import AdamW
from tqdm.auto import tqdm



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn_v2(weights="DEFAULT").to(device)

in_feats = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_feats, 2) # 1 Klasse + Hintergrund
model = model.to(device)


# Adam-Optimierers mit Weight Decay
# Fügt einen kleinen Strafterm hinzu, der große Gewichtswerte im Modell bestraft
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

EPOCHS = 10
loss_per_epoch = []

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    start = time.time()

    # tqdm() zeigt Fortschrittsbalken an
    for (imgs, targets) in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        # Alle Bilder im Batch auf GPU schieben
        imgs = [im.to(device) for im in imgs]
        # Alle Targets im Batch auf GPU schieben
        targets = [{k: v.to(device) for k,v in t.items()} for t in targets]
        # Gradienten aller Modellparameter auf null zurücksetzen
        optimizer.zero_grad()
        # Gesamten Verlust berechnen
        loss = sum(model(imgs, targets).values())
        # Backward-Pass mit skaliertem Verlust
        loss.backward()
        # Modellparameter aktualisieren
        optimizer.step()
        # Skalierungsfaktor aktualisieren, damit kleine Gradienten nicht zu klein werden
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    loss_per_epoch.append(avg_loss)
    dur = time.time() - start
    print(f"Epoch {epoch+1}/{EPOCHS} - train loss: {avg_loss:.4f} ({dur:.1f}s)")

# Modell speichern
torch.save(model.state_dict(), "last_model.pth")

Epoch 1/10:  21%|██        | 25/118 [13:50<51:30, 33.23s/it]  


KeyboardInterrupt: 

Loss

In [ ]:
plt.figure()
plt.plot(loss_per_epoch, marker='o')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training loss per epoch")
plt.savefig("metrics/loss_per_epoch.png")
plt.show()

Evaluation

In [ ]:
import torch
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.functional as F
from PIL import Image
import matplotlib.pyplot as plt

# Modell laden
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = fasterrcnn_resnet50_fpn_v2(weights=None)
in_feats = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_feats, 2)
model.load_state_dict(torch.load("models/best.pth", map_location=device))
model.to(device)
model.eval()

# Kein Gradient nötig beim Evaluieren
with torch.no_grad():
    for imgs, targets in test_loader:
        imgs = [im.to(device) for im in imgs]
        preds = model(imgs)  # Liste aus Dictionaries [{boxes, labels, scores}, ...]

        break  # Nur erste Batch (zum Anzeigen)

Ergebnisse anzeigen

In [ ]:
# Erstes Bild + Vorhersage
img = (imgs[0].cpu() * 255).byte()
pred = preds[0]

# Nur Boxen mit hoher Konfidenz zeigen
keep = pred["scores"] > 0.5
boxes = pred["boxes"][keep]
labels = pred["labels"][keep]

# Bounding Boxes zeichnen
img_boxed = draw_bounding_boxes(img, boxes, colors="red", width=2)
plt.imshow(F.to_pil_image(img_boxed))
plt.axis("off")
plt.show()
